In [ ]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import xgboost as xgb 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
trainPath = "train_final.csv"
testPath = "test_final.csv"
validationPath = "validation_final.csv"
sourcePath = "source.csv"

train = "train.csv"

train_df = pd.read_csv(trainPath)
test_df = pd.read_csv(testPath)
combine = [train_df,test_df]

In [ ]:
def submit(pred):
  submission = pd.DataFrame({
        "Prediction": pred
    })
  submission.to_csv('submission.csv', index=False)

In [ ]:
# preview the data
train_df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income>50K
0,53,Self-emp-not-inc,93449,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,India,1
1,33,Self-emp-not-inc,123424,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,1
2,47,Private,144844,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0
3,40,Private,114580,HS-grad,9,Divorced,Craft-repair,Other-relative,White,Female,0,0,40,Vietnam,0
4,39,Private,115618,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,50,United-States,0


## Mapping discrete variable

In [ ]:
workclass_mapping = {"Self-emp-not-inc": 1, "Federal-gov": 2, "Private": 3, "Local-gov": 4, "State-gov": 5,"Never-worked":6,"Self-emp-inc": 7,"Without-pay":8,"?":0}
race_mapping = {"White":1 ,"Black":2,"Asian-Pac-Islander":3,"Other":4,"Amer-Indian-Eskimo":5,"?":6}
education_mapping = {"Prof-school": 1,
                     "Bachelors": 2,
                     "HS-grad": 3,
                     "Doctorate": 4,
                     "Masters": 5,
                     "Some-college":6,
                     "11th": 7,
                     "12th":8,
                     "5th-6th":9,
                     "Assoc-acdm":10,
                     "9th":11,
                     "Preschool":12,
                     "Assoc-voc":13,
                     "7th-8th":14,
                     "10th":15,
                     "1st-4th":16
                     }
marital_mapping = {"Married-civ-spouse": 1,
                     "Divorced": 2,
                     "Never-married": 3,
                     "Married-spouse-absent": 4,
                     "Widowed": 5,
                     "Separated":6,
                     "Married-AF-spouse": 7,
                     }
occupation_mapping = {"Prof-specialty": 1,
                     "Exec-managerial": 2,
                     "Transport-moving": 3,
                     "Other-service": 4,
                     "Machine-op-inspct": 5,
                     "Adm-clerical":6,
                     "Sales": 7,
                     "Handlers-cleaners":8,
                      "Craft-repair":9,
                      "Farming-fishing":10,
                      "Tech-support":11,
                      "Priv-house-serv":12,
                      "Protective-serv":13,
                      "Armed-Forces":14,
                      "?":0
                     }
sex_mapping = {"Male":1,
               "Female":2
              }
# classfied countries list into sevearl categories:
# Highly developed countries: 
# Middle level developed countries
# Other developed cpuntries
# developing countries
native_country_mapping = {
"United-States":1,
"Ireland":1,
"Hong":1,
"Canada":1,
"Italy":1,
"Germany":1,
"England":1,
"Japan":1,
"France":1,
"Outlying-US(Guam-USVI-etc)":1,
"Scotland":1,

"Greece":2,
"Poland":2,
"Taiwan":2,
"Trinadad&Tobago":2,
"Portugal":2,
"Hungary":2, 

"China": 3,
"Dominican-Republic":3,
"Thailand":3,
"Mexico":3,

"Honduras":4,
"India":4,
"Vietnam":4,
"Guatemala":4,
"Iran":4,
"Laos":4,
"Philippines":4,
"Haiti":4,
"Cuba":4,
"Jamaica":4,
"Peru":4,
"Cambodia":4,
"Yugoslavia":4,
"El-Salvador":4,
"Columbia":4,
"Ecuador":4,
"South":4,
"Puerto-Rico":4,
"Nicaragua":4,
"?":4


}

In [ ]:
for dataset in combine:

    dataset['sex'] = dataset['sex'].fillna("Male")
    dataset['marital.status'] = dataset['marital.status'].fillna("Married-civ-spouse")
    dataset['education'] = dataset['education'].fillna("HS-grad")
    dataset["workclass"] = dataset["workclass"].fillna("Private")
    dataset["occupation"] =dataset["occupation"].fillna("Prof-specialty")
    dataset["native.country"] =dataset["native.country"].fillna("United-States")
    dataset["race"] =dataset["race"].fillna("White")

    dataset['sex'] = dataset['sex'].map(sex_mapping)
    dataset['native.country'] = dataset['native.country'].map(native_country_mapping)
    dataset['marital.status'] = dataset['marital.status'].map(marital_mapping)
    dataset['occupation'] = dataset['occupation'].map(occupation_mapping)
    dataset['education'] = dataset['education'].map(education_mapping)
    dataset['workclass'] = dataset['workclass'].map(workclass_mapping)
    dataset['race'] = dataset['race'].map(race_mapping)

    
train_df.head()


,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income>50K
0,53,1,93449,1,15,1,1,Husband,3,1,0,0,40,4,1
1,33,1,123424,2,13,1,2,Husband,1,1,0,0,40,1,1
2,47,3,144844,3,9,1,9,Husband,1,1,0,0,40,1,0
3,40,3,114580,3,9,2,9,Other-relative,1,2,0,0,40,4,0
4,39,3,115618,3,9,1,3,Husband,1,1,0,0,50,1,0


## Mapping countious variable

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
for dataset in combine:
  dataset['net.capital'] = dataset['capital.gain'].astype(float) + dataset['capital.loss'].astype(float)
  dataset['net.capital'] = scaler.fit(dataset['net.capital'].values.reshape(-1,1)).transform(dataset['net.capital'].values.reshape(-1,1))
  dataset['age'] = scaler.fit(dataset['age'].values.reshape(-1,1)).transform(dataset['age'].values.reshape(-1,1))
  #dataset['education.num'] = scaler.fit(dataset['education.num'].values.reshape(-1,1)).transform(dataset['education.num'].values.reshape(-1,1))
  dataset['hours.per.week'] = scaler.fit(dataset['hours.per.week'].values.reshape(-1,1)).transform(dataset['hours.per.week'].values.reshape(-1,1))
  dataset['fnlwgt'] = dataset['fnlwgt'].apply(lambda x: np.log1p(x))

train_df = train_df.drop(['capital.gain','capital.loss','relationship'],axis=1)
test_df = test_df.drop(['capital.gain','capital.loss','relationship'],axis=1)




In [ ]:

train_df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,race,sex,hours.per.week,native.country,income>50K,net.capital
0,1.042815,1,11.445182,1,15,1,1,3,1,-0.042502,4,1,-0.158595
1,-0.415003,1,11.723389,2,13,1,2,1,1,-0.042502,1,1,-0.158595
2,0.605470,3,11.883419,3,9,1,9,1,1,-0.042502,1,0,-0.158595
3,0.095233,3,11.649037,3,9,2,9,1,2,-0.042502,4,0,-0.158595
4,0.022343,3,11.658056,3,9,1,3,1,1,0.763076,1,0,-0.158595


In [ ]:
def submit(pred):
  submission = pd.DataFrame({
        "ID": test_df["ID"],
        "Prediction": pred
    })
  submission.to_csv('submission.csv', index=False)

## Apply machine learning models

In [ ]:
X_train = train_df.drop("income>50K", axis=1)
Y_train = train_df["income>50K"]

X_test  = test_df.drop("ID", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

X_test =X_test.fillna(1)

## Other machine learning models

In [ ]:
gbt = GradientBoostingClassifier(n_estimators=200,learning_rate=0.1,max_depth=5)
gbt.fit(X_train, Y_train)

GradientBoostingClassifier(max_depth=5, n_estimators=200)

In [ ]:
prediction = gbt.predict_proba(X_test)[:,1]
new_test_data = pd.read_csv("test_final.csv")
idx=new_test_data.iloc[:,0].to_numpy()
submissions = pd.DataFrame({'ID':idx, 'Prediction':prediction})
submit(prediction)

In [ ]:
prediction


array([0.16271546, 0.03331402, 0.15018692, ..., 0.99115375, 0.17693543,
       0.02183909])

In [ ]:
np.any(np.isnan(X_test))

False

In [ ]:

# Logistic regression
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
print(acc_log)


83.94


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# Support Vector Machines
svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
print(acc_svc)


84.29


In [ ]:

# best neighbor=9
knn = KNeighborsClassifier(n_neighbors = 9)
knn.fit(X_train, Y_train)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
print("k neighbor：",acc_knn)



k neighbor： 86.39


In [ ]:
# Gaussian Naive Bayes
gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
print(acc_gaussian)



82.16


In [ ]:
# Linear SVC

linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
print(acc_linear_svc)



83.22


/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
print(acc_sgd)

82.24


## Random forest and decision tree

In [ ]:
# Random Forest
# best maxp_depth = 12
for i in range(3,20):
  random_forest = RandomForestClassifier(n_estimators=120,max_depth=i,min_samples_split = 7,min_samples_leaf=3,
                                          oob_score = True)
  random_forest.fit(X_train, Y_train)
  Y_pred = random_forest.predict(X_test)
  submit(Y_pred)
  random_forest.score(X_train, Y_train)
  acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)

  print("depth：",i,acc_random_forest)

depth： 3 84.04
depth： 4 84.64
depth： 5 84.84
depth： 6 85.34
depth： 7 85.58
depth： 8 85.96
depth： 9 86.62
depth： 10 87.18
depth： 11 87.84
depth： 12 88.26
depth： 13 88.79
depth： 14 89.27
depth： 15 89.65
depth： 16 90.13
depth： 17 90.5
depth： 18 90.76
depth： 19 91.04


In [ ]:
# Decision Tree
# Best max depth = 9
for i in range(1,10):

  decision_tree = DecisionTreeClassifier(max_depth=i)
  decision_tree.fit(X_train, Y_train)
  Y_pred = decision_tree.predict(X_test)
  acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
  
  print("depth：",i,acc_decision_tree)

depth： 1 75.94
depth： 2 82.78
depth： 3 84.36
depth： 4 84.6
depth： 5 85.02
depth： 6 85.1
depth： 7 85.9
depth： 8 86.42
depth： 9 86.96


## Final solution Gradient Boosting classifier

In [ ]:
#GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
gbt = GradientBoostingClassifier(n_estimators=200,learning_rate=0.1,max_depth=5)
gbt.fit(X_train, Y_train)


prediction = gbt.predict_proba(X_test)[:,1]
new_test_data = pd.read_csv("test_final.csv")
idx=new_test_data.iloc[:,0].to_numpy()
submissions = pd.DataFrame({'ID':idx, 'Prediction':prediction})
submit(prediction)